In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import os
import random
import json
import glob

In [2]:
ROWS_PER_FRAME = 543
data_dir = "/kaggle/input/asl-signs"
landmark_fimes_dir = "/kaggle/input/asl-signs/train_landmark_files"

In [3]:
def seed_it_all(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


seed_it_all()

In [4]:
def load_relevant_data_subset(pq_path):
    data_columns = ["type","x", "y", "z"]
    data_return=pd.DataFrame(columns=["xl","yl","zl","xr","yr","zr"])
    data = pd.read_parquet(pq_path, columns=data_columns)
    data1 = data[data.type == "right_hand"]
    data2 = data[data.type == "left_hand"]
    data_return["xr"] = data1["x"]
    data_return["yr"] = data1["y"]
    data_return["zr"] = data1["z"]
    data_return["xl"] = data2["x"]
    data_return["yl"] = data2["y"]
    data_return["zl"] = data2["z"]
    n_frames = int(len(data_return) / 21)
    data_return = data_return.values.reshape(n_frames, 21, 6)
    return data_return.astype(np.float32)

In [5]:
def read_json(path):
    with open(path, "r") as file:
        json_data = json.load(file)
    return json_data

In [6]:
train_df = pd.read_csv(data_dir + "/train.csv")
train_df["path"] = data_dir + "/" + train_df["path"]
display(train_df)

,path,participant_id,sequence_id,sign
0,/kaggle/input/asl-signs/train_landmark_files/2...,26734,1000035562,blow
1,/kaggle/input/asl-signs/train_landmark_files/2...,28656,1000106739,wait
2,/kaggle/input/asl-signs/train_landmark_files/1...,16069,100015657,cloud
3,/kaggle/input/asl-signs/train_landmark_files/2...,25571,1000210073,bird
4,/kaggle/input/asl-signs/train_landmark_files/6...,62590,1000240708,owie
...,...,...,...,...
94472,/kaggle/input/asl-signs/train_landmark_files/5...,53618,999786174,white
94473,/kaggle/input/asl-signs/train_landmark_files/2...,26734,999799849,have
94474,/kaggle/input/asl-signs/train_landmark_files/2...,25571,999833418,flower
94475,/kaggle/input/asl-signs/train_landmark_files/2...,29302,999895257,room


In [27]:
s2p_map = read_json(os.path.join(data_dir, "sign_to_prediction_index_map.json"))
p2s_map = {v: k for k, v in s2p_map.items()}

encoder = lambda x: s2p_map.get(x)
decoder = lambda x: p2s_map.get(x)

train_df["label"] = train_df["sign"].map(encoder)
# print(f"shape = {train_df.shape}")
# p2s_map
# arr=train_df["label"].astype(np.float32)



In [40]:
sample_path = train_df.path[0]
sample = pd.read_parquet(sample_path)

print(f"Sample shape = {sample.shape}")
print(f"Number of Frames = {int(len(sample) / ROWS_PER_FRAME)}")
# ROWS_PER_FRAME = 543 i.e. one frame is represented by 543 row in our dataset, including the face, both hands and pose
# n_frame can also be found : 20->42=>23frames i.e. sample.frame.max() -> sample.frame.min() : sample.nunique()

display(sample)
display(sample.iloc[468:489])
display(sample.iloc[522:543])

Sample shape = (12489, 7)
Number of Frames = 23


,frame,row_id,type,landmark_index,x,y,z
0,20,20-face-0,face,0,0.494400,0.380470,-0.030626
1,20,20-face-1,face,1,0.496017,0.350735,-0.057565
2,20,20-face-2,face,2,0.500818,0.359343,-0.030283
3,20,20-face-3,face,3,0.489788,0.321780,-0.040622
4,20,20-face-4,face,4,0.495304,0.341821,-0.061152
...,...,...,...,...,...,...,...
12484,42,42-right_hand-16,right_hand,16,0.001660,0.549574,-0.145409
12485,42,42-right_hand-17,right_hand,17,0.042694,0.693116,-0.085307
12486,42,42-right_hand-18,right_hand,18,0.006723,0.665044,-0.114017
12487,42,42-right_hand-19,right_hand,19,-0.014755,0.643799,-0.123488


,frame,row_id,type,landmark_index,x,y,z
468,20,20-left_hand-0,left_hand,0,NaN,NaN,NaN
469,20,20-left_hand-1,left_hand,1,NaN,NaN,NaN
470,20,20-left_hand-2,left_hand,2,NaN,NaN,NaN
471,20,20-left_hand-3,left_hand,3,NaN,NaN,NaN
472,20,20-left_hand-4,left_hand,4,NaN,NaN,NaN
473,20,20-left_hand-5,left_hand,5,NaN,NaN,NaN
474,20,20-left_hand-6,left_hand,6,NaN,NaN,NaN
475,20,20-left_hand-7,left_hand,7,NaN,NaN,NaN
476,20,20-left_hand-8,left_hand,8,NaN,NaN,NaN
477,20,20-left_hand-9,left_hand,9,NaN,NaN,NaN


,frame,row_id,type,landmark_index,x,y,z
522,20,20-right_hand-0,right_hand,0,0.317082,0.576334,2.273533e-07
523,20,20-right_hand-1,right_hand,1,0.378220,0.522858,2.524794e-02
524,20,20-right_hand-2,right_hand,2,0.400223,0.473187,3.187834e-02
525,20,20-right_hand-3,right_hand,3,0.422157,0.436533,2.407637e-02
526,20,20-right_hand-4,right_hand,4,0.446464,0.403041,1.425959e-02
527,20,20-right_hand-5,right_hand,5,0.350601,0.440231,6.016464e-02
528,20,20-right_hand-6,right_hand,6,0.376617,0.397908,4.689403e-02
529,20,20-right_hand-7,right_hand,7,0.415079,0.385037,2.567031e-02
530,20,20-right_hand-8,right_hand,8,0.447915,0.381096,1.004762e-02
531,20,20-right_hand-9,right_hand,9,0.327356,0.440892,2.936660e-02


In [114]:
sample_data_np = load_relevant_data_subset(sample_path)
print(f"shape = {sample_data_np.shape} = (n_frames, row_per_frame, xyz) \n")
sample_data_np[0]

shape = (23, 21, 6) = (n_frames, row_per_frame, xyz) 



array([[           nan,            nan,            nan,  3.1708199e-01,
         5.7633448e-01,  2.2735328e-07],
       [           nan,            nan,            nan,  3.7822029e-01,
         5.2285826e-01,  2.5247939e-02],
       [           nan,            nan,            nan,  4.0022323e-01,
         4.7318691e-01,  3.1878337e-02],
       [           nan,            nan,            nan,  4.2215687e-01,
         4.3653315e-01,  2.4076371e-02],
       [           nan,            nan,            nan,  4.4646373e-01,
         4.0304130e-01,  1.4259585e-02],
       [           nan,            nan,            nan,  3.5060054e-01,
         4.4023085e-01,  6.0164642e-02],
       [           nan,            nan,            nan,  3.7661672e-01,
         3.9790779e-01,  4.6894025e-02],
       [           nan,            nan,            nan,  4.1507930e-01,
         3.8503700e-01,  2.5670307e-02],
       [           nan,            nan,            nan,  4.4791484e-01,
         3.8109562e-01, 

In [186]:
sample_path_hand = train_df.path[5]  # 5 : empirical choice
sample_for_hand = pd.read_parquet(sample_path_hand)

print(f"Number of Frames = {int(len(sample_for_hand) / ROWS_PER_FRAME)}")
print(f"First frame indice is {sample_for_hand.frame.min()}")
print(f"Last frame indice is {sample_for_hand.frame.max()}")
print(f"Sample signe is : {train_df.sign[5]}")

right_hand_sample = sample_for_hand[sample_for_hand.type == "right_hand"]
left_hand_sample = sample_for_hand[sample_for_hand.type == "left_hand"]
right_hand_sample.head(21)

Number of Frames = 30
First frame indice is 20
Last frame indice is 49
Sample signe is : duck


,frame,row_id,type,landmark_index,x,y,z
522,20,20-right_hand-0,right_hand,0,0.255993,0.540393,7.167730e-07
523,20,20-right_hand-1,right_hand,1,0.320184,0.544032,-4.895598e-02
524,20,20-right_hand-2,right_hand,2,0.385502,0.532575,-8.121844e-02
525,20,20-right_hand-3,right_hand,3,0.437874,0.532974,-1.135144e-01
526,20,20-right_hand-4,right_hand,4,0.484941,0.544218,-1.464274e-01
527,20,20-right_hand-5,right_hand,5,0.405232,0.440939,-6.005625e-02
528,20,20-right_hand-6,right_hand,6,0.440280,0.386434,-1.016259e-01
529,20,20-right_hand-7,right_hand,7,0.458284,0.360205,-1.214742e-01
530,20,20-right_hand-8,right_hand,8,0.474230,0.338766,-1.289929e-01
531,20,20-right_hand-9,right_hand,9,0.356082,0.426459,-6.341226e-02


In [48]:
class FeatureGen(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, x):
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        x = np.mean(x, axis=0)
        return x


feature_converter = FeatureGen()

In [115]:
data_lenght_experiment = len(train_df)
data_lenght_experiment

94477

In [137]:
def convert_row(row):
    x = load_relevant_data_subset(os.path.join("/kaggle/input/asl-signs", row.path))
    x = feature_converter(x)
    return x, row.label
def convert_and_save_data():
    np_features = np.zeros((data_lenght_experiment, 21, 6))
    np_labels = np.zeros(data_lenght_experiment)

    print(f"Total data to processe : {data_lenght_experiment}")
    for index, row in tqdm(train_df.iterrows()):
        if index > data_lenght_experiment - 1:
            break

        data = load_relevant_data_subset(row.path)
        feature, label = convert_row(row)
        np_features[index, :, :] = feature
        np_labels[index] = label

    np.save("features.npy", np_features)
    np.save("labels.npy", np_labels)

In [138]:
try:
    features = np.load("/kaggle/working/feature.npy")
    labels = np.load("/kaggle/working/label.npy")
except:
    convert_and_save_data()

Total data to processe : 94477


94477it [1:11:08, 22.13it/s]


In [148]:
!ls /kaggle

input  lib  working


In [154]:
features = np.load("features.npy")
labels = np.load("labels.npy")

array([ 25., 232.,  48., ...,  86., 188., 105.])

In [173]:
def get_model(n_labels=250, learning_rate=0.001):
    inputs = layers.Input(shape=(21,6))
    x = layers.Flatten()(inputs)
    x = layers.Dense(320, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = layers.Dense(290, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.Dropout(0.6)(x)
    x = layers.Dense(264, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
#     x = layers.Dense(250, activation="relu")(x)
    # x = layers.Dense(8, activation="relu")(x)
    x = layers.Flatten()(x)
    output = layers.Dense(n_labels, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=output)

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=["accuracy"],
    )

    return model

In [174]:
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "./ASL_model",
    save_best_only=True,
    restore_best_weights=True,
    monitor="val_accuracy",
    mode="max",
    verbose=False,
)

cb_list = [checkpoint_callback]

X_train, X_val, y_train, y_val = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

model = get_model()
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 21, 6)]           0         
                                                                 
 flatten_6 (Flatten)         (None, 126)               0         
                                                                 
 dense_17 (Dense)            (None, 320)               40640     
                                                                 
 batch_normalization_12 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 activation_12 (Activation)  (None, 320)               0         
                                                                 
 dropout_12 (Dropout)        (None, 320)               0         
                                                           

In [185]:
try:
    model = tf.keras.models.load_model("./ASL_mode")
except:
    history = model.fit(
        features,
        labels,
        validation_data=(X_val, y_val),
        epochs=50,
        callbacks=cb_list,
        batch_size=64,
    )

Epoch 1/50
1477/1477 [==============================] - 15s 8ms/step - loss: 3.9608 - accuracy: 0.1796 - val_loss: 3.7349 - val_accuracy: 0.2360
Epoch 2/50
1477/1477 [==============================] - 13s 9ms/step - loss: 3.9594 - accuracy: 0.1799 - val_loss: 3.7103 - val_accuracy: 0.2450
Epoch 3/50
1477/1477 [==============================] - 12s 8ms/step - loss: 3.9543 - accuracy: 0.1813 - val_loss: 3.7412 - val_accuracy: 0.2308
Epoch 4/50
1477/1477 [==============================] - 15s 10ms/step - loss: 3.9503 - accuracy: 0.1816 - val_loss: 3.6881 - val_accuracy: 0.2457
Epoch 5/50
1477/1477 [==============================] - 14s 10ms/step - loss: 3.9435 - accuracy: 0.1812 - val_loss: 3.6983 - val_accuracy: 0.2472
Epoch 6/50
1477/1477 [==============================] - 11s 8ms/step - loss: 3.9450 - accuracy: 0.1818 - val_loss: 3.6895 - val_accuracy: 0.2426
Epoch 7/50
1477/1477 [==============================] - 13s 9ms/step - loss: 3.9347 - accuracy: 0.1836 - val_loss: 3.6501 - val_

In [176]:
model = tf.keras.models.load_model("./ASL_model")
score = model.evaluate(X_val, y_val)

591/591 [==============================] - 2s 4ms/step - loss: 3.7322 - accuracy: 0.2401


In [177]:
def get_inference_model(model):
    inputs = tf.keras.Input((21, 6), dtype=tf.float32, name="inputs")
    x = tf.where(tf.math.is_nan(inputs), tf.zeros_like(inputs), inputs)
    x = tf.reduce_mean(x, axis=0, keepdims=True)
    x = model(x)
    output = tf.keras.layers.Activation(activation="linear", name="outputs")(x)
    inference_model = tf.keras.Model(inputs=inputs, outputs=output)
    inference_model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"]
    )
    return inference_model

In [178]:
inference_model = get_inference_model(model)
inference_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, 21, 6)]      0           []                               
                                                                                                  
 tf.math.is_nan_2 (TFOpLambda)  (None, 21, 6)        0           ['inputs[0][0]']                 
                                                                                                  
 tf.zeros_like_2 (TFOpLambda)   (None, 21, 6)        0           ['inputs[0][0]']                 
                                                                                                  
 tf.where_2 (TFOpLambda)        (None, 21, 6)        0           ['tf.math.is_nan_2[0][0]',       
                                                                  'tf.zeros_like_2[0][0]',  

In [182]:
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
tflite_model = converter.convert()
model_path = "model.tflite"
# Save the model.
with open(model_path, "wb") as f:
    f.write(tflite_model)

In [184]:
!ls /kaggle/working/model.tflite

/kaggle/working/model.tflite


In [180]:
model.save("mask_detector.model", save_format="h5")